In [ ]:
#torch

from torch.nn import Parameter
import torch
import torch.optim
import torch.optim as optim
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F

import torchvision.models as models
import torchvision.transforms as transforms

import torch.utils.data as Data
import torch.backends.cudnn as cudnn


#torchnet???
#import torchnet as tnt


#numpy
import numpy as np

#pandas
import pandas as pd

#utility

import math
from urllib.request import urlretrieve

from PIL import Image
#from tqdm import tqdm
from tqdm.auto import tqdm
import random

import json
import os
import pickle
import shutil
import time

import urllib

In [ ]:
num_classes = 23

image_size = 224

change_class_style = {0: "classic",
         1: "preppy", 
         2: "manish",
         3: "tomboy", 
         4: "feminine",
         5: "romantic",
         6: "sexy",
         7: "hippie",
         8: "western",
         9: "oriental",
         10: "modern",
         11: "sophisticated",
         12: "avantgarde",
         13: "country",
         14: "resort",
         15: "genderless",
         16: "sporty",
         17: "retro",
         18: "hiphop",
         19: "kitsch",
         20: "punk",
         21: "street",
         22: "military"
         }

In [ ]:
class Warp(object):
    def __init__(self, size, interpolation=Image.BILINEAR):
        self.size = int(size)
        self.interpolation = interpolation

    def __call__(self, img):
        return img.resize((self.size, self.size), self.interpolation)

    def __str__(self):
        return self.__class__.__name__ + ' (size={size}, interpolation={interpolation})'.format(size=self.size,
                                                                                                interpolation=self.interpolation)
class MultiScaleCrop(object):

    def __init__(self, input_size, scales=None, max_distort=1, fix_crop=True, more_fix_crop=True):
        self.scales = scales if scales is not None else [1, 875, .75, .66]
        self.max_distort = max_distort
        self.fix_crop = fix_crop
        self.more_fix_crop = more_fix_crop
        self.input_size = input_size if not isinstance(input_size, int) else [input_size, input_size]
        self.interpolation = Image.BILINEAR

    def __call__(self, img):
        im_size = img.size
        crop_w, crop_h, offset_w, offset_h = self._sample_crop_size(im_size)
        crop_img_group = img.crop((offset_w, offset_h, offset_w + crop_w, offset_h + crop_h))
        ret_img_group = crop_img_group.resize((self.input_size[0], self.input_size[1]), self.interpolation)
        return ret_img_group

    def _sample_crop_size(self, im_size):
        image_w, image_h = im_size[0], im_size[1]

        # find a crop size
        base_size = min(image_w, image_h)
        crop_sizes = [int(base_size * x) for x in self.scales]
        crop_h = [self.input_size[1] if abs(x - self.input_size[1]) < 3 else x for x in crop_sizes]
        crop_w = [self.input_size[0] if abs(x - self.input_size[0]) < 3 else x for x in crop_sizes]

        pairs = []
        for i, h in enumerate(crop_h):
            for j, w in enumerate(crop_w):
                if abs(i - j) <= self.max_distort:
                    pairs.append((w, h))

        crop_pair = random.choice(pairs)
        if not self.fix_crop:
            w_offset = random.randint(0, image_w - crop_pair[0])
            h_offset = random.randint(0, image_h - crop_pair[1])
        else:
            w_offset, h_offset = self._sample_fix_offset(image_w, image_h, crop_pair[0], crop_pair[1])

        return crop_pair[0], crop_pair[1], w_offset, h_offset

    def _sample_fix_offset(self, image_w, image_h, crop_w, crop_h):
        offsets = self.fill_fix_offset(self.more_fix_crop, image_w, image_h, crop_w, crop_h)
        return random.choice(offsets)

    @staticmethod
    def fill_fix_offset(more_fix_crop, image_w, image_h, crop_w, crop_h):
        w_step = (image_w - crop_w) // 4
        h_step = (image_h - crop_h) // 4

        ret = list()
        ret.append((0, 0))  # upper left
        ret.append((4 * w_step, 0))  # upper right
        ret.append((0, 4 * h_step))  # lower left
        ret.append((4 * w_step, 4 * h_step))  # lower right
        ret.append((2 * w_step, 2 * h_step))  # center

        if more_fix_crop:
            ret.append((0, 2 * h_step))  # center left
            ret.append((4 * w_step, 2 * h_step))  # center right
            ret.append((2 * w_step, 4 * h_step))  # lower center
            ret.append((2 * w_step, 0 * h_step))  # upper center

            ret.append((1 * w_step, 1 * h_step))  # upper left quarter
            ret.append((3 * w_step, 1 * h_step))  # upper right quarter
            ret.append((1 * w_step, 3 * h_step))  # lower left quarter
            ret.append((3 * w_step, 3 * h_step))  # lower righ quarter

        return ret


    def __str__(self):
        return self.__class__.__name__


def download_url(url, destination=None, progress_bar=True):
    """Download a URL to a local file.

    Parameters
    ----------
    url : str
        The URL to download.
    destination : str, None
        The destination of the file. If None is given the file is saved to a temporary directory.
    progress_bar : bool
        Whether to show a command-line progress bar while downloading.

    Returns
    -------
    filename : str
        The location of the downloaded file.

    Notes
    -----
    Progress bar use/example adapted from tqdm documentation: https://github.com/tqdm/tqdm
    """

    def my_hook(t):
        last_b = [0]

        def inner(b=1, bsize=1, tsize=None):
            if tsize is not None:
                t.total = tsize
            if b > 0:
                t.update((b - last_b[0]) * bsize)
            last_b[0] = b

        return inner

    if progress_bar:
        with tqdm(unit='B', unit_scale=True, miniters=1, desc=url.split('/')[-1]) as t:
            filename, _ = urlretrieve(url, filename=destination, reporthook=my_hook(t))
    else:
        filename, _ = urlretrieve(url, filename=destination)


class AveragePrecisionMeter(object):
    """
    The APMeter measures the average precision per class.
    The APMeter is designed to operate on `NxK` Tensors `output` and
    `target`, and optionally a `Nx1` Tensor weight where (1) the `output`
    contains model output scores for `N` examples and `K` classes that ought to
    be higher when the model is more convinced that the example should be
    positively labeled, and smaller when the model believes the example should
    be negatively labeled (for instance, the output of a sigmoid function); (2)
    the `target` contains only values 0 (for negative examples) and 1
    (for positive examples); and (3) the `weight` ( > 0) represents weight for
    each sample.
    """

    def __init__(self, difficult_examples=False):
        super(AveragePrecisionMeter, self).__init__()
        self.reset()
        self.difficult_examples = difficult_examples

    def reset(self):
        """Resets the meter with empty member variables"""
        self.scores = torch.FloatTensor(torch.FloatStorage())
        self.targets = torch.LongTensor(torch.LongStorage())

    def add(self, output, target):
        """
        Args:
            output (Tensor): NxK tensor that for each of the N examples
                indicates the probability of the example belonging to each of
                the K classes, according to the model. The probabilities should
                sum to one over all classes
            target (Tensor): binary NxK tensort that encodes which of the K
                classes are associated with the N-th input
                    (eg: a row [0, 1, 0, 1] indicates that the example is
                         associated with classes 2 and 4)
            weight (optional, Tensor): Nx1 tensor representing the weight for
                each example (each weight > 0)
        """
        if not torch.is_tensor(output):
            output = torch.from_numpy(output)
        if not torch.is_tensor(target):
            target = torch.from_numpy(target)

        if output.dim() == 1:
            output = output.view(-1, 1)
        else:
            assert output.dim() == 2, \
                'wrong output size (should be 1D or 2D with one column \
                per class)'
        if target.dim() == 1:
            target = target.view(-1, 1)
        else:
            assert target.dim() == 2, \
                'wrong target size (should be 1D or 2D with one column \
                per class)'
        if self.scores.numel() > 0:
            assert target.size(1) == self.targets.size(1), \
                'dimensions for output should match previously added examples.'

        # make sure storage is of sufficient size
        if self.scores.storage().size() < self.scores.numel() + output.numel():
            new_size = math.ceil(self.scores.storage().size() * 1.5)
            self.scores.storage().resize_(int(new_size + output.numel()))
            self.targets.storage().resize_(int(new_size + output.numel()))

        # store scores and targets
        offset = self.scores.size(0) if self.scores.dim() > 0 else 0
        self.scores.resize_(offset + output.size(0), output.size(1))
        self.targets.resize_(offset + target.size(0), target.size(1))
        self.scores.narrow(0, offset, output.size(0)).copy_(output)
        self.targets.narrow(0, offset, target.size(0)).copy_(target)

    def value(self):
        """Returns the model's average precision for each class
        Return:
            ap (FloatTensor): 1xK tensor, with avg precision for each class k
        """

        if self.scores.numel() == 0:
            return 0
        ap = torch.zeros(self.scores.size(1))
        rg = torch.arange(1, self.scores.size(0)).float()
        # compute average precision for each class
        for k in range(self.scores.size(1)):
            # sort scores
            scores = self.scores[:, k]
            targets = self.targets[:, k]
            # compute average precision
            ap[k] = AveragePrecisionMeter.average_precision(scores, targets, self.difficult_examples)
        return ap

    @staticmethod
    def average_precision(output, target, difficult_examples=True):

        # sort examples
        sorted, indices = torch.sort(output, dim=0, descending=True)

        # Computes prec@i
        pos_count = 0.
        total_count = 0.
        precision_at_i = 0.
        for i in indices:
            label = target[i]
            if difficult_examples and label == 0:
                continue
            if label == 1:
                pos_count += 1
            total_count += 1
            if label == 1:
                precision_at_i += pos_count / total_count
        precision_at_i /= pos_count
        return precision_at_i

    def overall(self):
        if self.scores.numel() == 0:
            return 0
        scores = self.scores.cpu().numpy()
        targets = self.targets.cpu().numpy()
        targets[targets == -1] = 0
        return self.evaluation(scores, targets)

    def overall_topk(self, k):
        targets = self.targets.cpu().numpy()
        targets[targets == -1] = 0
        n, c = self.scores.size()
        scores = np.zeros((n, c)) - 1
        index = self.scores.topk(k, 1, True, True)[1].cpu().numpy()
        tmp = self.scores.cpu().numpy()
        for i in range(n):
            for ind in index[i]:
                scores[i, ind] = 1 if tmp[i, ind] >= 0 else -1
        return self.evaluation(scores, targets)


    def evaluation(self, scores_, targets_):
        n, n_class = scores_.shape
        Nc, Np, Ng = np.zeros(n_class), np.zeros(n_class), np.zeros(n_class)
        for k in range(n_class):
            scores = scores_[:, k]
            targets = targets_[:, k]
            targets[targets == -1] = 0
            Ng[k] = np.sum(targets == 1)
            Np[k] = np.sum(scores >= 0)
            Nc[k] = np.sum(targets * (scores >= 0))
        Np[Np == 0] = 1
        OP = np.sum(Nc) / np.sum(Np)
        OR = np.sum(Nc) / np.sum(Ng)
        OF1 = (2 * OP * OR) / (OP + OR)

        CP = np.sum(Nc / Np) / n_class
        CR = np.sum(Nc / Ng) / n_class
        CF1 = (2 * CP * CR) / (CP + CR)
        return OP, OR, OF1, CP, CR, CF1

def top_n_recall(preds, truths, n=1):
    best_n = np.argsort(preds, axis=1)[:,-n:]
    ts = np.argmax(truths, axis=1)
    successes = 0
    s_img_id = []
    w_img_id = []
    for i in range(ts.shape[0]):
        if ts[i] in best_n[i,:]:
            successes += 1
            s_img_id.append(i)
        else:
            w_img_id.append(i)

    return float(successes)/ts.shape[0], s_img_id, w_img_id, best_n

def gen_A(num_classes, t, adj_file):
    import pickle
    result = pickle.load(open(adj_file, 'rb'))
    _adj = result['adj']
    _nums = result['nums']
    _nums = _nums[:, np.newaxis]
    _adj = _adj / _nums
    _adj[_adj < t] = 0
    _adj[_adj >= t] = 1
    _adj = _adj * 0.25 / (_adj.sum(0, keepdims=True) + 1e-6)
    _adj = _adj + np.identity(num_classes, np.int)
    return _adj

def gen_adj(A):
    D = torch.pow(A.sum(1).float(), -0.5)
    D = torch.diag(D)
    adj = torch.matmul(torch.matmul(A, D).t(), D)
    return adj

In [ ]:
## Load word2vec
with open("/content/drive/MyDrive/pickle/custom_word2vec.pkl","rb") as fr:
    word2vec = pickle.load(fr)
print(word2vec)

[[ 0.2890625   0.06982422  0.16210938 ... -0.12451172  0.02148438
  -0.04370117]
 [ 0.03930664  0.10595703 -0.18359375 ... -0.21484375  0.16113281
   0.36132812]
 [-0.14550781  0.03637695  0.05175781 ... -0.0625     -0.07910156
   0.02819824]
 ...
 [ 0.05224609 -0.03808594  0.20410156 ... -0.265625    0.09570312
   0.14355469]
 [ 0.09814453  0.07568359  0.03088379 ... -0.04833984  0.21289062
  -0.13867188]
 [-0.03442383  0.21679688 -0.09472656 ... -0.04003906  0.15527344
   0.10009766]]


In [ ]:
class GraphConvolution(nn.Module):

    def __init__(self, in_features, out_features, bias=False):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.Tensor(1, 1, out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.matmul(input, self.weight)
        output = torch.matmul(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


class GCNResnet(nn.Module):
    def __init__(self, model, num_classes, in_channel=300, t=0, adj_file=None):
        super(GCNResnet, self).__init__()
        self.features = nn.Sequential(
            model.conv1,
            model.bn1,
            model.relu,
            model.maxpool,
            model.layer1,
            model.layer2,
            model.layer3,
            model.layer4,
        )
        self.num_classes = num_classes
        # self.pooling = nn.MaxPool2d(14, 14)
        self.pooling = nn.MaxPool2d(7, 7)

        self.gc1 = GraphConvolution(in_channel, 1024)
        self.gc2 = GraphConvolution(1024, 2048)
        self.relu = nn.LeakyReLU(0.2)

        _adj = gen_A(num_classes, t, adj_file)
        self.A = Parameter(torch.from_numpy(_adj).float())
        #         # image normalization
        self.image_normalization_mean = [0.485, 0.456, 0.406]
        self.image_normalization_std = [0.229, 0.224, 0.225]

    def forward(self, feature, inp):
        feature = self.features(feature)
        feature = self.pooling(feature)
        feature = feature.view(feature.size(0), -1)
        inp = inp[0]
        adj = gen_adj(self.A).detach()
        x = self.gc1(inp, adj)
        x = self.relu(x)
        x = self.gc2(x, adj)
        x = x.transpose(0, 1)
        x = torch.matmul(feature, x)
        return x

    def get_config_optim(self, lr, lrp):
        return [
                {'params': self.features.parameters(), 'lr': lr * lrp},
                {'params': self.gc1.parameters(), 'lr': lr},
                {'params': self.gc2.parameters(), 'lr': lr},
                ]


def gcn_resnet101(num_classes, t, pretrained=True, adj_file=None, in_channel=300):
    model = models.resnet101(pretrained=pretrained)
    return GCNResnet(model, num_classes, t=t, adj_file=adj_file, in_channel=in_channel)

In [ ]:
#load ImageNet pretrained model

adj = "/content/drive/MyDrive/pickle/adj_file.pkl"

model = gcn_resnet101(num_classes=num_classes, t=0.03, pretrained = True, adj_file=adj, in_channel = 300)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

<ipython-input-12-ac00a1f67dd8>:307: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _adj = _adj + np.identity(num_classes, np.int)


In [ ]:
#transform

normalize = transforms.Normalize(mean = model.image_normalization_mean, std = model.image_normalization_std)

val_transform = transforms.Compose([
    Warp(image_size),
    transforms.ToTensor(),
    normalize,
])

In [ ]:
resume = "/content/drive/MyDrive/stylemodel_save/style_best_model4.pth"

checkpoint = torch.load(resume, map_location=torch.device('cpu'))

model.load_state_dict(checkpoint)

<All keys matched successfully>

# data extraction

In [ ]:
with open("/content/drive/MyDrive/singledata/가방/wallet&pouch.json", 'r') as file:
    data = json.load(file)

In [ ]:
data

[{'id': 3771,
  'name': 'D.LAB 엘린 3단 중지갑 (4colors) [월렛in월렛 : 키링+카드지갑]',
  'url': 'https://www.brandi.co.kr/products/17218481',
  'imageUrl': 'https://image.brandi.me/cproduct/2021/04/28/SB000000000008366784_1619597393_image1_S.jpeg',
  'price': '26,660원',
  'seller': '디랩'},
 {'id': 3772,
  'name': 'D.Lab Lira Accordion Card Wallet-5colors',
  'url': 'https://www.brandi.co.kr/products/74301795',
  'imageUrl': 'https://image.brandi.me/cproduct/2022/12/05/SB000000000066777810_1670224464_image1_S.jpeg',
  'price': '26,980원',
  'seller': '디랩'},
 {'id': 3773,
  'name': 'D.LAB Leather simple card wallet - 13 color',
  'url': 'https://www.brandi.co.kr/products/4931085',
  'imageUrl': 'https://image.brandi.me/cproduct/2020/07/13/4931085_1594632505_image1_M.jpg',
  'price': '12,780원',
  'seller': '디랩'},
 {'id': 3774,
  'name': 'D.LAB [키링증정] flip simple card wallet - 6color',
  'url': 'https://www.brandi.co.kr/products/68888210',
  'imageUrl': 'https://image.brandi.me/cproduct/2022/11/14/SB000000

In [ ]:
url_list = []

for d in data:
    
    url_list.append(d['imageUrl'])

In [ ]:
print(len(data))

100


In [ ]:
print(len(url_list))

100


In [ ]:
inp = torch.tensor(word2vec).unsqueeze(0)

# inference

In [ ]:
model.eval()

for j,url in tqdm(enumerate(url_list)):

    img = Image.open(urllib.request.urlretrieve(url)[0]).convert("RGB")

    if val_transform is not None:
    
        img = val_transform(img)
    
    img = img.unsqueeze(0)

    feature_var = torch.autograd.Variable(img).float()
    inp_var = torch.autograd.Variable(inp).float().detach()  # one hot

    output = model(feature_var, inp_var)
    percentage_output = F.softmax(output, dim = 1)

    pred = output.cpu().detach().numpy()

    sorted_pred = np.argsort(pred,axis = 1)

    result = {}

    for i in range(22,-1,-1):
        
        result[23-i] = {
            "style":change_class_style[sorted_pred[0][i]],
            "score":round(percentage_output[0][sorted_pred[0][i]].item()*100,4)
        }
    
    data[j]['style_vector'] = result


0it [00:00, ?it/s]

In [ ]:
data

In [ ]:
with open("/content/drive/MyDrive/singledata/가방/wallet&pouch.json", 'w', encoding='utf-8') as file:
    json.dump(data, file)

In [ ]:
with open("/content/drive/MyDrive/singledata/가방/wallet&pouch.json", 'r') as file:
    data = json.load(file)

In [ ]:
data

# automatic

In [ ]:
inp = torch.tensor(word2vec).unsqueeze(0)

In [ ]:
json_list = os.listdir("/content/drive/MyDrive/singledata/니트류/")

print(json_list)

['v-neck.json', 'tutle-neck.json', 'cardigan.json', 'cashmere.json', 'skirt.json', 'onepiecedress.json']


In [ ]:
root_path = "/content/drive/MyDrive/singledata/패션소품/"

json_list = os.listdir("/content/drive/MyDrive/singledata/패션소품/")

for filename in tqdm(json_list):

    with open(root_path + filename, 'r') as file:
        data = json.load(file)

    url_list = []

    for d in data:
        
        url_list.append(d['imageUrl'])

    model.eval()

    for j,url in tqdm(enumerate(url_list)):

        img = Image.open(urllib.request.urlretrieve(url)[0]).convert("RGB")

        if val_transform is not None:
        
            img = val_transform(img)
        
        img = img.unsqueeze(0)

        feature_var = torch.autograd.Variable(img).float()
        inp_var = torch.autograd.Variable(inp).float().detach()  # one hot

        output = model(feature_var, inp_var)
        percentage_output = F.softmax(output, dim = 1)

        pred = output.cpu().detach().numpy()

        sorted_pred = np.argsort(pred,axis = 1)

        result = {}

        for i in range(22,-1,-1):
            
            result[23-i] = {
                "style":change_class_style[sorted_pred[0][i]],
                "score":round(percentage_output[0][sorted_pred[0][i]].item()*100,4)
            }
        
        data[j]['style_vector'] = result
    
    with open(root_path + filename, 'w', encoding='utf-8') as file:
        
        json.dump(data, file)

  0%|          | 0/7 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
import json

In [ ]:
with open("/content/drive/MyDrive/pseudo_label/가방/backpack.json", 'r') as file:
    data1 = json.load(file)

data1

In [ ]:
with open("/content/drive/MyDrive/singledata/가방/backpack.json", 'r') as file:
    data2 = json.load(file)

data2

In [ ]:
data1[1]

{'id': 3672,
 'name': '[해외] 나이키 스포츠웨어 퓨추라 럭스 토트백',
 'url': 'https://www.brandi.co.kr/products/76004555',
 'imageUrl': 'https://image.brandi.me/cproduct/2022/07/26/SB000000000068523111_1658860405_image1_S.jpeg',
 'price': '171,900원',
 'seller': '직구생활연구소',
 'style_vector': {'1': {'style': 'street', 'score': 16.6796},
  '2': {'style': 'military', 'score': 13.6613},
  '3': {'style': 'sporty', 'score': 11.947},
  '4': {'style': 'genderless', 'score': 11.8741},
  '5': {'style': 'modern', 'score': 6.712},
  '6': {'style': 'sexy', 'score': 5.1907},
  '7': {'style': 'avantgarde', 'score': 4.6201},
  '8': {'style': 'punk', 'score': 4.2761},
  '9': {'style': 'classic', 'score': 2.6925},
  '10': {'style': 'kitsch', 'score': 2.5633},
  '11': {'style': 'retro', 'score': 2.2439},
  '12': {'style': 'preppy', 'score': 2.0262},
  '13': {'style': 'western', 'score': 1.9877},
  '14': {'style': 'hippie', 'score': 1.9383},
  '15': {'style': 'tomboy', 'score': 1.8711},
  '16': {'style': 'feminine', 'score': 

In [ ]:
data2[1]

{'id': 3672,
 'name': '[해외] 나이키 스포츠웨어 퓨추라 럭스 토트백',
 'url': 'https://www.brandi.co.kr/products/76004555',
 'imageUrl': 'https://image.brandi.me/cproduct/2022/07/26/SB000000000068523111_1658860405_image1_S.jpeg',
 'price': '171,900원',
 'seller': '직구생활연구소',
 'style_vector': {'1': {'style': 'street', 'score': 93.4563},
  '2': {'style': 'modern', 'score': 3.5035},
  '3': {'style': 'sexy', 'score': 2.7783},
  '4': {'style': 'sporty', 'score': 0.1245},
  '5': {'style': 'classic', 'score': 0.0591},
  '6': {'style': 'retro', 'score': 0.03},
  '7': {'style': 'military', 'score': 0.0185},
  '8': {'style': 'genderless', 'score': 0.012},
  '9': {'style': 'kitsch', 'score': 0.0064},
  '10': {'style': 'preppy', 'score': 0.0038},
  '11': {'style': 'feminine', 'score': 0.0019},
  '12': {'style': 'punk', 'score': 0.0017},
  '13': {'style': 'sophisticated', 'score': 0.0016},
  '14': {'style': 'hippie', 'score': 0.0006},
  '15': {'style': 'manish', 'score': 0.0004},
  '16': {'style': 'hiphop', 'score': 0.

In [ ]:
data1[2]

{'id': 3673,
 'name': '세인트스코트 코린 퀼팅 백팩',
 'url': 'https://www.brandi.co.kr/products/94548069',
 'imageUrl': 'https://image.brandi.me/cproduct/2023/03/08/SB000000000087463339_1678253011_image1_S.jpeg',
 'price': '99,230원',
 'seller': '세인트스코트핸드백',
 'style_vector': {'1': {'style': 'street', 'score': 29.9825},
  '2': {'style': 'modern', 'score': 9.9359},
  '3': {'style': 'classic', 'score': 9.0059},
  '4': {'style': 'sophisticated', 'score': 8.824},
  '5': {'style': 'feminine', 'score': 6.26},
  '6': {'style': 'genderless', 'score': 5.2001},
  '7': {'style': 'romantic', 'score': 4.4524},
  '8': {'style': 'sexy', 'score': 3.3973},
  '9': {'style': 'country', 'score': 3.3618},
  '10': {'style': 'manish', 'score': 3.1709},
  '11': {'style': 'tomboy', 'score': 2.9807},
  '12': {'style': 'retro', 'score': 2.679},
  '13': {'style': 'sporty', 'score': 2.1573},
  '14': {'style': 'resort', 'score': 1.7237},
  '15': {'style': 'kitsch', 'score': 1.5797},
  '16': {'style': 'avantgarde', 'score': 1.553

In [ ]:
data2[2]

{'id': 3673,
 'name': '세인트스코트 코린 퀼팅 백팩',
 'url': 'https://www.brandi.co.kr/products/94548069',
 'imageUrl': 'https://image.brandi.me/cproduct/2023/03/08/SB000000000087463339_1678253011_image1_S.jpeg',
 'price': '99,230원',
 'seller': '세인트스코트핸드백',
 'style_vector': {'1': {'style': 'street', 'score': 98.564},
  '2': {'style': 'sexy', 'score': 0.4022},
  '3': {'style': 'sophisticated', 'score': 0.3431},
  '4': {'style': 'modern', 'score': 0.3032},
  '5': {'style': 'classic', 'score': 0.2925},
  '6': {'style': 'sporty', 'score': 0.0279},
  '7': {'style': 'feminine', 'score': 0.0225},
  '8': {'style': 'retro', 'score': 0.0149},
  '9': {'style': 'preppy', 'score': 0.0087},
  '10': {'style': 'romantic', 'score': 0.0068},
  '11': {'style': 'kitsch', 'score': 0.0064},
  '12': {'style': 'country', 'score': 0.0033},
  '13': {'style': 'genderless', 'score': 0.0013},
  '14': {'style': 'military', 'score': 0.0011},
  '15': {'style': 'manish', 'score': 0.0008},
  '16': {'style': 'resort', 'score': 0.00

In [ ]:
data1[3]

{'id': 3674,
 'name': '[3/30 예약배송] Multi Bucket Bag Black',
 'url': 'https://www.brandi.co.kr/products/22474430',
 'imageUrl': 'https://image.brandi.me/cproduct/2020/11/02/22474430_1604295711_image1_M.jpg',
 'price': '78,400원',
 'seller': '사비',
 'style_vector': {'1': {'style': 'street', 'score': 18.6885},
  '2': {'style': 'classic', 'score': 14.634},
  '3': {'style': 'country', 'score': 13.5755},
  '4': {'style': 'genderless', 'score': 9.713},
  '5': {'style': 'modern', 'score': 8.2902},
  '6': {'style': 'feminine', 'score': 7.0647},
  '7': {'style': 'western', 'score': 3.8313},
  '8': {'style': 'sophisticated', 'score': 3.6515},
  '9': {'style': 'romantic', 'score': 3.1195},
  '10': {'style': 'manish', 'score': 2.7686},
  '11': {'style': 'tomboy', 'score': 2.3938},
  '12': {'style': 'avantgarde', 'score': 2.356},
  '13': {'style': 'resort', 'score': 2.1743},
  '14': {'style': 'sporty', 'score': 1.741},
  '15': {'style': 'retro', 'score': 1.5783},
  '16': {'style': 'military', 'score':

In [ ]:
data2[3]

{'id': 3674,
 'name': '[3/30 예약배송] Multi Bucket Bag Black',
 'url': 'https://www.brandi.co.kr/products/22474430',
 'imageUrl': 'https://image.brandi.me/cproduct/2020/11/02/22474430_1604295711_image1_M.jpg',
 'price': '78,400원',
 'seller': '사비',
 'style_vector': {'1': {'style': 'classic', 'score': 57.0851},
  '2': {'style': 'street', 'score': 39.2008},
  '3': {'style': 'feminine', 'score': 1.3727},
  '4': {'style': 'sexy', 'score': 0.7895},
  '5': {'style': 'modern', 'score': 0.7513},
  '6': {'style': 'country', 'score': 0.4495},
  '7': {'style': 'military', 'score': 0.1712},
  '8': {'style': 'genderless', 'score': 0.0728},
  '9': {'style': 'preppy', 'score': 0.0335},
  '10': {'style': 'sporty', 'score': 0.0319},
  '11': {'style': 'retro', 'score': 0.0159},
  '12': {'style': 'sophisticated', 'score': 0.0095},
  '13': {'style': 'romantic', 'score': 0.0051},
  '14': {'style': 'western', 'score': 0.0047},
  '15': {'style': 'kitsch', 'score': 0.0025},
  '16': {'style': 'manish', 'score': 0.

In [ ]:
data1[4]

{'id': 3675,
 'name': 'Daily backpack _ Cocoa',
 'url': 'https://www.brandi.co.kr/products/87029358',
 'imageUrl': 'https://image.brandi.me/cproduct/2023/01/17/SN000000000079765000_1673977811_image1_S.jpeg',
 'price': '46,230원',
 'seller': '마스마룰즈',
 'style_vector': {'1': {'style': 'genderless', 'score': 31.7639},
  '2': {'style': 'street', 'score': 26.2194},
  '3': {'style': 'sporty', 'score': 8.2362},
  '4': {'style': 'tomboy', 'score': 6.3959},
  '5': {'style': 'military', 'score': 5.8937},
  '6': {'style': 'classic', 'score': 5.1431},
  '7': {'style': 'modern', 'score': 2.7261},
  '8': {'style': 'country', 'score': 2.5273},
  '9': {'style': 'preppy', 'score': 1.9599},
  '10': {'style': 'kitsch', 'score': 1.444},
  '11': {'style': 'manish', 'score': 1.4387},
  '12': {'style': 'retro', 'score': 1.0791},
  '13': {'style': 'avantgarde', 'score': 0.9327},
  '14': {'style': 'feminine', 'score': 0.8182},
  '15': {'style': 'resort', 'score': 0.7774},
  '16': {'style': 'western', 'score': 0.

In [ ]:
data2[4]

{'id': 3675,
 'name': 'Daily backpack _ Cocoa',
 'url': 'https://www.brandi.co.kr/products/87029358',
 'imageUrl': 'https://image.brandi.me/cproduct/2023/01/17/SN000000000079765000_1673977811_image1_S.jpeg',
 'price': '46,230원',
 'seller': '마스마룰즈',
 'style_vector': {'1': {'style': 'street', 'score': 74.3408},
  '2': {'style': 'classic', 'score': 15.6792},
  '3': {'style': 'military', 'score': 6.2821},
  '4': {'style': 'feminine', 'score': 2.2613},
  '5': {'style': 'modern', 'score': 0.6867},
  '6': {'style': 'genderless', 'score': 0.2946},
  '7': {'style': 'country', 'score': 0.1255},
  '8': {'style': 'sexy', 'score': 0.0968},
  '9': {'style': 'sporty', 'score': 0.0674},
  '10': {'style': 'preppy', 'score': 0.0644},
  '11': {'style': 'romantic', 'score': 0.0385},
  '12': {'style': 'sophisticated', 'score': 0.0343},
  '13': {'style': 'retro', 'score': 0.0084},
  '14': {'style': 'manish', 'score': 0.0072},
  '15': {'style': 'western', 'score': 0.0038},
  '16': {'style': 'kitsch', 'score'